<a href="https://colab.research.google.com/github/ErikaNat/Inteligencia-Artificial/blob/main/EjercicioDeep9Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#==========================================
# 0) Imports y Configuracion  hacer prediccion, implementarla en esta bnueva red neuronal, con mismas librerias, para red secienciasl y base de preprocesamiento, la digerencia es que  no hay prediccion, se entrena,

#==========================================
import os
import numpy as np
import pandas as pd
import tensorflow as tf

# 1. Importar el framework de red neuronal (Keras/TensorFlow)
from tensorflow.keras.models import Sequential  # Modelo secuencial (capas una tras otra)
from tensorflow.keras.layers import Dense, Dropout  # Capas densas (completamente conectadas) y Dropout
from tensorflow.keras.optimizers import Adam, RMSprop  # Optimizadores comunes

# 2. Importar herramientas para preparar los datos
from sklearn.model_selection import train_test_split  # Para dividir en train/validation
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # Para escalar/normalizar los datos (CRUCIAL en regresión)

# 3. Importar herramientas para evaluar el modelo
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt  # Para graficar resultados

#Reproductibilidad
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow:", tf.__version__)
print("Keras:", tf.keras.__version__)

TensorFlow: 2.19.0
Keras: 3.10.0


In [ ]:
#==========================================
#1) Cargar dataset
#==========================================

df = pd.read_csv("insurance.csv")
display(df.head())

#Mirada Rapida
print("Shape bruto:" , df.shape)
print("Cols", list(df.columns))

#============================
#2) Seleccion de variables and Target Mapping
#============================
# Seoleccion de columnas especificas
feature_cols = ["age", "sex", "bmi", "smoker", 'children', "region"]
x_train, x_val, y_train, y_val = train_test_split(df[feature_cols], df["charges"], test_size=0.2, random_state=SEED)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


Shape bruto: (1338, 7)
Cols ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Identificar columnas numericas
categorical_features = ['sex', 'smoker', 'region']
numerical_features = ['age', 'bmi', 'children']


# Crear columna transf
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)


print("Imput dims:", x_train.shape[1])


X_train_scaled = preprocessor.fit_transform(x_train)
X_val_scaled = preprocessor.transform(x_val)

print("Shape after preprocessing:", X_train_scaled.shape)

Imput dims: 6
Shape after preprocessing: (1070, 11)


In [ ]:
#===================================
#6) Definir y compilar  el modelo
#===================================
def build_model(input_dim: int) -> tf.keras.Model:
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(64, activation="relu"), # decrescer de neuronas
        tf.keras.layers.Dropout(0.2), # Dropout
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation="linear") # para regersion
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # Usar a Adam como optimizador
        loss="mse",
        metrics=["mae", "mse"]
    )
    return model

model = build_model(X_train_scaled.shape[1])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ================================
# 7) Callbacks (buenoas prácticas)
# ================================
cbs = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=20, restore_best_weights=True), # Monitor validation loss for regression
    tf.keras.callbacks.ModelCheckpoint("insurance_best_model.keras", monitor="val_loss", mode="min", save_best_only=True), # Save best model based on validation loss
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, min_lr=0.0001) # Reduce learning rate on validation loss plateau
]

In [ ]:
# ====================
# 8) Entrenamiento
# ====================
hist = model.fit(
    X_train_scaled, y_train, # Usa escala de entrenamiento
    validation_data=(X_val_scaled, y_val), # Validacion
    epochs=200,
    batch_size=32,
    callbacks=cbs,
    verbose=1
)

Epoch 1/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 318429152.0000 - mae: 13298.4150 - mse: 318429152.0000 - val_loss: 323380128.0000 - val_mae: 12966.6455 - val_mse: 323380128.0000 - learning_rate: 0.0010
Epoch 2/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 318365216.0000 - mae: 13296.0811 - mse: 318365216.0000 - val_loss: 323261088.0000 - val_mae: 12962.2344 - val_mse: 323261088.0000 - learning_rate: 0.0010
Epoch 3/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 318209408.0000 - mae: 13290.3975 - mse: 318209408.0000 - val_loss: 322954144.0000 - val_mae: 12950.9307 - val_mse: 322954144.0000 - learning_rate: 0.0010
Epoch 4/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 317803680.0000 - mae: 13275.7666 - mse: 317803680.0000 - val_loss: 322238496.0000 - val_mae: 12924.7207 - val_mse: 322238496.0000 - learning_rate: 0.0010
Epoch 5/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 316893728.0000 - mae: 13243.1055 - mse: 316893728.0000 - val_loss: 320822368.0000 - val_

In [ ]:
# ==========================
# 9) Evaluación  en test
# ==========================
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np # Import numpy for sqrt

# 1. UsaR EL MODELO ENTRNADO
y_pred = model.predict(X_val_scaled).ravel()


mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse) # Calculate RMSE
r2 = r2_score(y_val, y_pred)

print(f"Mean Squared Error on Validation Data: {mse:.2f}")
print(f"Root Mean Squared Error on Validation Data: {rmse:.2f}") # Print RMSE
print(f"Mean Absolute Error on Validation Data: {mae:.2f}")
print(f"R-squared on Validation Data: {r2:.2f}")



9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Mean Squared Error on Validation Data: 28301036.78
Root Mean Squared Error on Validation Data: 5319.87
Mean Absolute Error on Validation Data: 3558.12
R-squared on Validation Data: 0.82


In [ ]:
# =============================================================
# 10) Interferencia robusta: función predict_one(dict de entrada)
# =============================================================
def predict_one(sample: dict) -> float:
    """
    Recibe un diccionario 'crudo' con las llaves esperadas:
    age, sex, bmi, children, smoker, region.
    Devuelve la predicción del cargo (charges).
    """
    # Convertir a DataFrame con columnas en orden esperado
    # Ellaves con x
    expected_cols = ["age", "sex", "bmi", "smoker", 'children', "region"]
    # Reordenar diccionario
    ordered_sample = {col: sample.get(col) for col in expected_cols}
    s = pd.DataFrame([ordered_sample])

    # aplicar mismo proessamiento
    # Usar el procesador
    s_proc = preprocessor.transform(s) # trabsformar el nuevo ejemplo

    # Predecir
    prediction = model.predict(s_proc).item() # prediccion
    return prediction

# Ejemplo de inferencia
sample = {
    "age": 30,
    "sex": "female",
    "bmi": 25.0,
    "children": 1,
    "smoker": "no",
    "region": "southwest"
}



# Ejemplo resultado
predicted_charge = predict_one(sample)
print(f"Predicted charge for the sample: {predicted_charge:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted charge for the sample: 4671.18
